In [1]:
import pathlib
from itertools import count
from typing import Union, List

import numpy as np
import xarray as xr
from matplotlib import pyplot as plt

from qec_util import Layout
from qec_util.layouts import plot as plot_layout

from surface_sim import Setup
from surface_sim.models import CircuitNoiseModel
from surface_sim.experiments import memory_exp
from surface_sim.util import sample_experiment

In [2]:
EXP_DIR : str = pathlib.Path.cwd()

CONFIG_DIR = EXP_DIR / "config"
if not CONFIG_DIR.exists():
    raise ValueError("Layout directory does not exist.")

In [3]:
LAYOUT_FILE = "d3_rotated_layout.yaml"
SETUP_FILE = "gen_circ_noise.yaml"

layout = Layout.from_yaml(CONFIG_DIR / LAYOUT_FILE)
setup = Setup.from_yaml(CONFIG_DIR / SETUP_FILE)
model = CircuitNoiseModel(setup, layout)

# Generate the training data

In [4]:
DATASET_TYPE : str = "dev" # Possible types are "train", "dev" and "test"

# Fixed parameters
ROOT_SEED : Union[int, None] = 4028 # Initial seed for the RNG
NUM_ROUNDS : int = 20 # Number of rounds
NUM_SHOTS : int = 10000 # Number of shots
ROT_BASIS : bool = False  # In the z-basis
MEAS_RESET : bool = False # No resets following measurements

# Variable parameters
LOG_STATES : List[int] = [0, 1] # Logical state(s)

In [5]:
basis = "X" if ROT_BASIS else "Z"
seed_sequence = np.random.SeedSequence(ROOT_SEED)

num_experiments = len(LOG_STATES)
seeds = iter(seed_sequence.generate_state(num_experiments, dtype="uint64"))
datasets = []

for log_state in LOG_STATES:
    exp_name = f"surf-code_d{layout.distance}_b{basis}_s{log_state}_n{NUM_SHOTS}_r{NUM_ROUNDS}"

    exp_folder = EXP_DIR / DATASET_TYPE / exp_name
    exp_folder.mkdir(parents=True, exist_ok=True)

    experiment = memory_exp(
        model=model,
        num_rounds=NUM_ROUNDS,
        log_state=log_state,
        rot_basis=ROT_BASIS,
        meas_reset=MEAS_RESET,
    )

    experiment.to_file(exp_folder / "circuit.stim")

    seed = next(seeds)
    dataset = sample_experiment(
        layout,
        experiment,
        seed=seed,
        num_shots=NUM_SHOTS,
        num_rounds=NUM_ROUNDS,
    )

    # assign these as coordinate for merging datasets later on. Add here any otther relevant parameters
    dataset = dataset.assign_coords(
        log_state=log_state,
        rot_basis=ROT_BASIS,
        meas_reset=MEAS_RESET,
    )

    dataset.to_netcdf(exp_folder / "measurements.nc")

    error_model = experiment.detector_error_model(
        decompose_errors=True,
        allow_gauge_detectors=True,
    )
    error_model.to_file(exp_folder / "detector_error_model.dem")
